<a href="https://colab.research.google.com/github/bonghyunyang/llm_with_rag_chatbot/blob/main/LLM_With_LAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# 필요한 패키지를 설치합니다.
# langchain: 언어 모델 체인을 쉽게 구축할 수 있는 라이브러리
#  - 챗봇이나 질문 답변 시스템을 만들 때 유용하게 사용할 수 있어요.
# openai: OpenAI의 GPT 모델을 사용하기 위한 라이브러리
#  - OpenAI에서 만든 인공지능을 사용하기 위해 필요해요.
# faiss-cpu: 대용량 벡터 검색을 위한 Facebook AI의 라이브러리 (CPU 버전)
#  - 데이터를 빠르게 검색하고 찾아주는 역할을 해요. CPU에서 사용할 수 있는 버전이에요.
# pandas: 데이터 처리를 위한 데이터프레임 라이브러리
#  - 표 형태의 데이터를 쉽게 다룰 수 있도록 도와주는 도구에요.
!pip install langchain openai faiss-cpu pandas python-dotenv langchain-openai langchain-community

In [47]:
import os
import openai
from langchain_openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'sk-로 시작하는 그것'

In [42]:
import os

import pandas as pd

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

# CSV 파일 예시 (위스키 데이터 생성)
# 데이터 사전(dictionary)을 만듭니다. 여기에는 위스키의 이름, 도수, 예상 가격, 추천 대상이 들어가요.
data_dict = {
    '명칭': ['글렌피딕 12년산', '매캘란 18년산', '히비키 하모니', '야먀자키 12년산'],
    '도수': ['40%', '43%', '43%', '43%'],
    '예상금액': [50000, 300000, 120000, 150000],
    '추천대상': ['남성', '남성', '여성', '여성']
}

chat_data = pd.DataFrame(data_dict)
csv_path = "data.csv"
chat_data.to_csv(csv_path, index=False)

In [44]:
def setup_chatbot(api_key):
    # OpenAI API 키 설정
    os.environ['OPENAI_API_KEY'] = api_key

    # 데이터 준비
    chat_data = pd.read_csv(csv_path)
    data = chat_data[['명칭', '도수', '예상금액', '추천대상']]

    # 임베딩 및 벡터 저장소 설정
    embeddings = OpenAIEmbeddings()

    # 문서 생성
    documents = []
    for _, row in data.iterrows():
        content = f"위스키명: {row['명칭']}, 도수: {row['도수']}, 가격: {row['예상금액']}원, 추천대상: {row['추천대상']}"
        metadata = row.to_dict()
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)

    # FAISS 벡터 저장소 생성
    vector_store = FAISS.from_documents(documents, embeddings)

    # 메모리 설정
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )

    # ChatOpenAI 모델 설정
    llm = ChatOpenAI(temperature=0.7)

    # 대화형 검색 체인 생성
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(),
        memory=memory,
        verbose=True
    )

    return qa_chain

In [49]:
def chatbot_conversation(qa_chain, question):
    return qa_chain.invoke({"question": question})["answer"]

# 메인 실행 코드
if __name__ == "__main__":
    api_key = os.environ['OPENAI_API_KEY']
    qa_chain = setup_chatbot(api_key)

    print("안녕하세요! 위스키에 대해 어떤 것이든 물어보세요!")
    while True:
        user_input = input("메세지: ")
        if user_input.lower() in ["종료", "그만", "exit"]:
            print("대화를 종료합니다. 감사합니다!")
            break

        try:
            response = chatbot_conversation(qa_chain, user_input)
            print(f"Bot: {response}")
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")

안녕하세요! 위스키에 대해 어떤 것이든 물어보세요!


KeyboardInterrupt: Interrupted by user